In [2]:
import json
from py2neo import Graph
from py2neo.data import Node, Relationship

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")

#graph.delete_all()

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

Connected to graph database with 0 nodes and 0 relationships!


In [ ]:
# Constrain one id per Patent
print("Creating uniqueness constraint (and also index) on Patent titles's...", end=" ", flush=True)
query = """CREATE CONSTRAINT ON (n:Patent) ASSERT n.title IS UNIQUE;"""
graph.run(query).evaluate()
print("Done.")